In [4]:
import pandas as pd
import numpy as np

f1 = pd.read_csv("/home/yg2895/media/protein/yg2895/ASO-lib-20250102/DE_analyse/hek-dna-rna.csv")
f2 = pd.read_csv("../counts-aso.csv")

In [5]:
f1.rename(columns={'Unnamed: 0': 'Name'}, inplace=True)

f1.head()

,Name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,AAGAB|ENST00000261880.10|1,2659.245449,0.086427,0.106517,0.811394,4.171394e-01,4.454345e-01
1,AAGAB|ENST00000261880.10|2,2489.885932,0.511739,0.143115,3.575721,3.492642e-04,4.715106e-04
2,AAGAB|ENST00000261880.10|3,6038.426170,0.867749,0.072504,11.968271,5.210315e-33,2.027268e-32
3,AAGAB|ENST00000261880.10|4,2022.130110,0.539307,0.119582,4.509938,6.484647e-06,9.592057e-06
4,AAGAB|ENST00000261880.10|5,3272.032753,0.430862,0.092700,4.647896,3.353381e-06,5.036167e-06


In [6]:
merged_df = pd.merge(f1, f2, on='Name')
merged_df.head()

,Name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,Seq,hek-gDNA1,hek-gDNA2,...,hek-RNA2,hek-RNA3,sk-gDNA1,sk-gDNA2,sk-gDNA3,sk-RNA1,sk-RNA2,sk-RNA3,aso-lib1,aso-lib2
0,AAGAB|ENST00000261880.10|1,2659.245449,0.086427,0.106517,0.811394,4.171394e-01,4.454345e-01,ATTATTCATACTAGGGTTTGACCAACAAAG,2931,2511,...,2164,2430,3041,2212,1318,1660,1599,2036,3000,2635
1,AAGAB|ENST00000261880.10|2,2489.885932,0.511739,0.143115,3.575721,3.492642e-04,4.715106e-04,TTAGTTTTTAGTAGGAGGTTAAGGAGAAAT,2201,1890,...,2269,2796,3213,2573,1350,1891,1828,1708,3358,2859
2,AAGAB|ENST00000261880.10|3,6038.426170,0.867749,0.072504,11.968271,5.210315e-33,2.027268e-32,TAAATCTCAGAGGTCTGTGTGATTTACTCA,4923,4555,...,6474,6490,6847,5955,6049,3490,3270,3675,7867,6851
3,AAGAB|ENST00000261880.10|4,2022.130110,0.539307,0.119582,4.509938,6.484647e-06,9.592057e-06,ATTTGGGTTTTCTCCATTCAAGGAGCTTGT,1870,1612,...,1965,2236,2174,1716,1201,1500,1300,1170,2041,1822
4,AAGAB|ENST00000261880.10|5,3272.032753,0.430862,0.092700,4.647896,3.353381e-06,5.036167e-06,GAAAATGCCTCTGGGCATTCTTTTGAAGTA,3262,2828,...,3202,3289,3404,2529,2081,2491,2280,2227,3110,2710


### Filter: mean DNA counts > 100

In [5]:
# Specify the columns to calculate the mean
columns_to_check = ['hek-gDNA1', 'hek-gDNA2', 'hek-gDNA3']

# Calculate the mean for the specified columns
merged_df['hek_gDNA_mean'] = merged_df[columns_to_check].mean(axis=1)

# Filter rows where the mean of specified columns is >= 100
filtered_df = merged_df[merged_df['hek_gDNA_mean'] >= 100]

# Drop the intermediate column if not needed
filtered_df = filtered_df.drop(columns=['hek_gDNA_mean'])

# Output the filtered DataFrame
len(filtered_df)

# filtered_df.to_csv('filerted_hek_df100.csv', index=False)

11750

### All the gDNA counts > 100

In [7]:
# Specify the columns to calculate the mean
columns_to_check = ['hek-gDNA1', 'hek-gDNA2', 'hek-gDNA3']

filtered_df = merged_df[(merged_df[columns_to_check] > 100).all(axis=1)]

# Output the filtered DataFrame
len(filtered_df)

filtered_df.to_csv('filerted_hek_df100.csv', index=False)

### kmer analysis

In [1]:
## Remove the other columns, keep only name and log2FC
import pandas as pd

df = pd.read_csv("filerted_hek_df100.csv")
df = df[["Name","log2FoldChange"]]

df.to_csv("hek_nor_filter_100.txt", sep='\t', index=False)

In [2]:
## Remove the other columns, keep only name and log2FC
import pandas as pd

df = pd.read_csv("filerted_sk_df100.csv")
df = df[["Name","log2FoldChange"]]

df.to_csv("sk_nor_filter_100.txt", sep='\t', index=False)

In [ ]:
## kmer analysis
MatchScan -t /home/xw2629/xuebing/haploinsufficiency/utr3-pool-20240415/target-seq.fa -s /home/yg2895/media/protein/yg2895/ASO-lib-20250102/DE_analyse/hek_nor_filter_100.txt -k 6

### Remove polyA motif (AATAAA|ATTAAA)

In [6]:
seq_df = pd.read_csv("/home/yg2895/media/scratch/yang/haploinsufficiency/haplo-utr3-oligo-pool_new.csv")
merged_df = pd.merge(filtered_df, seq_df, on='Name')
merged_df.head()

,Name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,Seq,hek-gDNA1,hek-gDNA2,...,sk-gDNA1,sk-gDNA2,sk-gDNA3,sk-RNA1,sk-RNA2,sk-RNA3,aso-lib1,aso-lib2,Sequence,sequence
0,AAGAB|ENST00000261880.10|1,2659.245449,0.086427,0.106517,0.811394,4.171394e-01,4.454345e-01,ATTATTCATACTAGGGTTTGACCAACAAAG,2931,2511,...,3041,2212,1318,1660,1599,2036,3000,2635,GTTCAGCGTCGACCAATTATTCATACTAGGGTTTGACCAACAAAGA...,GTTCAGCGTCGACCAATTATTCATACTAGGGTTTGACCAACAAAGA...
1,AAGAB|ENST00000261880.10|2,2489.885932,0.511739,0.143115,3.575721,3.492642e-04,4.715106e-04,TTAGTTTTTAGTAGGAGGTTAAGGAGAAAT,2201,1890,...,3213,2573,1350,1891,1828,1708,3358,2859,GTTCAGCGTCGACCATTAGTTTTTAGTAGGAGGTTAAGGAGAAATC...,GTTCAGCGTCGACCATTAGTTTTTAGTAGGAGGTTAAGGAGAAATC...
2,AAGAB|ENST00000261880.10|3,6038.426170,0.867749,0.072504,11.968271,5.210315e-33,2.027268e-32,TAAATCTCAGAGGTCTGTGTGATTTACTCA,4923,4555,...,6847,5955,6049,3490,3270,3675,7867,6851,GTTCAGCGTCGACCATAAATCTCAGAGGTCTGTGTGATTTACTCAA...,GTTCAGCGTCGACCATAAATCTCAGAGGTCTGTGTGATTTACTCAA...
3,AAGAB|ENST00000261880.10|4,2022.130110,0.539307,0.119582,4.509938,6.484647e-06,9.592057e-06,ATTTGGGTTTTCTCCATTCAAGGAGCTTGT,1870,1612,...,2174,1716,1201,1500,1300,1170,2041,1822,GTTCAGCGTCGACCAATTTGGGTTTTCTCCATTCAAGGAGCTTGTA...,GTTCAGCGTCGACCAATTTGGGTTTTCTCCATTCAAGGAGCTTGTA...
4,AAGAB|ENST00000261880.10|5,3272.032753,0.430862,0.092700,4.647896,3.353381e-06,5.036167e-06,GAAAATGCCTCTGGGCATTCTTTTGAAGTA,3262,2828,...,3404,2529,2081,2491,2280,2227,3110,2710,GTTCAGCGTCGACCAGAAAATGCCTCTGGGCATTCTTTTGAAGTAT...,GTTCAGCGTCGACCAGAAAATGCCTCTGGGCATTCTTTTGAAGTAT...


In [7]:
polyA_rm_df = merged_df[~merged_df['sequence'].str.contains('AATAAA|ATTAAA')]

# Get the length of the filtered DataFrame
print(len(polyA_rm_df))

polyA_rm_df.to_csv('fitered_no_polyA_hek.csv',index=False)

7666


## Sk cell, filter based on DNA, remove AATAAA|ATTAAA

In [8]:
import pandas as pd
import numpy as np

f1 = pd.read_csv("/home/yg2895/media/protein/yg2895/ASO-lib-20250102/DE_analyse/sk-dna-rna.csv")
f2 = pd.read_csv("../counts-aso.csv")

f1.rename(columns={'Unnamed: 0': 'Name'}, inplace=True)

f1.head()

,Name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,AAGAB|ENST00000261880.10|1,1948.960707,0.542017,0.243431,2.226575,0.025976,0.033377
1,AAGAB|ENST00000261880.10|2,2050.560240,0.457778,0.255737,1.790031,0.073449,0.089355
2,AAGAB|ENST00000261880.10|3,4616.950869,-0.036766,0.117904,-0.311828,0.755172,0.779177
3,AAGAB|ENST00000261880.10|4,1488.596011,0.483780,0.196646,2.460154,0.013888,0.018414
4,AAGAB|ENST00000261880.10|5,2507.701739,0.644686,0.145270,4.437847,0.000009,0.000016


In [9]:
merged_df = pd.merge(f1, f2, on='Name')
merged_df.head()

,Name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,Seq,hek-gDNA1,hek-gDNA2,...,hek-RNA2,hek-RNA3,sk-gDNA1,sk-gDNA2,sk-gDNA3,sk-RNA1,sk-RNA2,sk-RNA3,aso-lib1,aso-lib2
0,AAGAB|ENST00000261880.10|1,1948.960707,0.542017,0.243431,2.226575,0.025976,0.033377,ATTATTCATACTAGGGTTTGACCAACAAAG,2931,2511,...,2164,2430,3041,2212,1318,1660,1599,2036,3000,2635
1,AAGAB|ENST00000261880.10|2,2050.560240,0.457778,0.255737,1.790031,0.073449,0.089355,TTAGTTTTTAGTAGGAGGTTAAGGAGAAAT,2201,1890,...,2269,2796,3213,2573,1350,1891,1828,1708,3358,2859
2,AAGAB|ENST00000261880.10|3,4616.950869,-0.036766,0.117904,-0.311828,0.755172,0.779177,TAAATCTCAGAGGTCTGTGTGATTTACTCA,4923,4555,...,6474,6490,6847,5955,6049,3490,3270,3675,7867,6851
3,AAGAB|ENST00000261880.10|4,1488.596011,0.483780,0.196646,2.460154,0.013888,0.018414,ATTTGGGTTTTCTCCATTCAAGGAGCTTGT,1870,1612,...,1965,2236,2174,1716,1201,1500,1300,1170,2041,1822
4,AAGAB|ENST00000261880.10|5,2507.701739,0.644686,0.145270,4.437847,0.000009,0.000016,GAAAATGCCTCTGGGCATTCTTTTGAAGTA,3262,2828,...,3202,3289,3404,2529,2081,2491,2280,2227,3110,2710


In [10]:
# Specify the columns to calculate the mean
columns_to_check = ['sk-gDNA1', 'sk-gDNA2', 'sk-gDNA3']

filtered_df = merged_df[(merged_df[columns_to_check] > 100).all(axis=1)]

len(filtered_df)

filtered_df.to_csv('filerted_sk_df100.csv', index=False)

In [23]:
seq_df = pd.read_csv("/home/yg2895/media/scratch/yang/haploinsufficiency/haplo-utr3-oligo-pool_new.csv")
merged_df = pd.merge(filtered_df, seq_df, on='Name')
merged_df.head()

,Name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,Seq,hek-gDNA1,hek-gDNA2,...,sk-gDNA1,sk-gDNA2,sk-gDNA3,sk-RNA1,sk-RNA2,sk-RNA3,aso-lib1,aso-lib2,Sequence,sequence
0,AAGAB|ENST00000261880.10|1,1948.960707,0.542017,0.243431,2.226575,0.025976,0.033377,ATTATTCATACTAGGGTTTGACCAACAAAG,2931,2511,...,3041,2212,1318,1660,1599,2036,3000,2635,GTTCAGCGTCGACCAATTATTCATACTAGGGTTTGACCAACAAAGA...,GTTCAGCGTCGACCAATTATTCATACTAGGGTTTGACCAACAAAGA...
1,AAGAB|ENST00000261880.10|2,2050.560240,0.457778,0.255737,1.790031,0.073449,0.089355,TTAGTTTTTAGTAGGAGGTTAAGGAGAAAT,2201,1890,...,3213,2573,1350,1891,1828,1708,3358,2859,GTTCAGCGTCGACCATTAGTTTTTAGTAGGAGGTTAAGGAGAAATC...,GTTCAGCGTCGACCATTAGTTTTTAGTAGGAGGTTAAGGAGAAATC...
2,AAGAB|ENST00000261880.10|3,4616.950869,-0.036766,0.117904,-0.311828,0.755172,0.779177,TAAATCTCAGAGGTCTGTGTGATTTACTCA,4923,4555,...,6847,5955,6049,3490,3270,3675,7867,6851,GTTCAGCGTCGACCATAAATCTCAGAGGTCTGTGTGATTTACTCAA...,GTTCAGCGTCGACCATAAATCTCAGAGGTCTGTGTGATTTACTCAA...
3,AAGAB|ENST00000261880.10|4,1488.596011,0.483780,0.196646,2.460154,0.013888,0.018414,ATTTGGGTTTTCTCCATTCAAGGAGCTTGT,1870,1612,...,2174,1716,1201,1500,1300,1170,2041,1822,GTTCAGCGTCGACCAATTTGGGTTTTCTCCATTCAAGGAGCTTGTA...,GTTCAGCGTCGACCAATTTGGGTTTTCTCCATTCAAGGAGCTTGTA...
4,AAGAB|ENST00000261880.10|5,2507.701739,0.644686,0.145270,4.437847,0.000009,0.000016,GAAAATGCCTCTGGGCATTCTTTTGAAGTA,3262,2828,...,3404,2529,2081,2491,2280,2227,3110,2710,GTTCAGCGTCGACCAGAAAATGCCTCTGGGCATTCTTTTGAAGTAT...,GTTCAGCGTCGACCAGAAAATGCCTCTGGGCATTCTTTTGAAGTAT...


In [24]:
polyA_rm_df = merged_df[~merged_df['sequence'].str.contains('AATAAA|ATTAAA')]

# Get the length of the filtered DataFrame
print(len(polyA_rm_df))

polyA_rm_df.to_csv('fitered_no_polyA_sk.csv',index=False)

7629


### Remove G4 structure

### Using Quadron through R

In [8]:
import pandas as pd
no_G4 = pd.read_csv('/home/yg2895/media/protein/yg2895/ASO-lib-20250102/quadron_pre/utr3-oligo-no-g4.csv')

In [9]:
filter_df = pd.merge(no_G4, polyA_rm_df, on='Name')
len(filter_df)

7081

In [10]:
filter_df.head()

,Name,sequence_x,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,Seq,hek-gDNA1,...,sk-gDNA1,sk-gDNA2,sk-gDNA3,sk-RNA1,sk-RNA2,sk-RNA3,aso-lib1,aso-lib2,Sequence,sequence_y
0,AAGAB|ENST00000261880.10|1,GTTCAGCGTCGACCAATTATTCATACTAGGGTTTGACCAACAAAGA...,2659.245449,0.086427,0.106517,0.811394,4.171394e-01,4.454345e-01,ATTATTCATACTAGGGTTTGACCAACAAAG,2931,...,3041,2212,1318,1660,1599,2036,3000,2635,GTTCAGCGTCGACCAATTATTCATACTAGGGTTTGACCAACAAAGA...,GTTCAGCGTCGACCAATTATTCATACTAGGGTTTGACCAACAAAGA...
1,AAGAB|ENST00000261880.10|2,GTTCAGCGTCGACCATTAGTTTTTAGTAGGAGGTTAAGGAGAAATC...,2489.885932,0.511739,0.143115,3.575721,3.492642e-04,4.715106e-04,TTAGTTTTTAGTAGGAGGTTAAGGAGAAAT,2201,...,3213,2573,1350,1891,1828,1708,3358,2859,GTTCAGCGTCGACCATTAGTTTTTAGTAGGAGGTTAAGGAGAAATC...,GTTCAGCGTCGACCATTAGTTTTTAGTAGGAGGTTAAGGAGAAATC...
2,AAGAB|ENST00000261880.10|3,GTTCAGCGTCGACCATAAATCTCAGAGGTCTGTGTGATTTACTCAA...,6038.426170,0.867749,0.072504,11.968271,5.210315e-33,2.027268e-32,TAAATCTCAGAGGTCTGTGTGATTTACTCA,4923,...,6847,5955,6049,3490,3270,3675,7867,6851,GTTCAGCGTCGACCATAAATCTCAGAGGTCTGTGTGATTTACTCAA...,GTTCAGCGTCGACCATAAATCTCAGAGGTCTGTGTGATTTACTCAA...
3,AAGAB|ENST00000261880.10|10,GTTCAGCGTCGACCAAGTACAGTGGATGGAAGGTGCCCGCACACCA...,1746.292863,0.658290,0.106247,6.195863,5.796659e-10,1.030479e-09,AGTACAGTGGATGGAAGGTGCCCGCACACC,1631,...,2174,2129,3386,1606,1556,1752,3766,3224,GTTCAGCGTCGACCAAGTACAGTGGATGGAAGGTGCCCGCACACCA...,GTTCAGCGTCGACCAAGTACAGTGGATGGAAGGTGCCCGCACACCA...
4,AAGAB|ENST00000261880.10|11,GTTCAGCGTCGACCAGACACTACCTTATAAGCACTGCAGAACCTGA...,3245.827395,-0.844369,0.103300,-8.173917,2.985350e-16,6.707519e-16,GACACTACCTTATAAGCACTGCAGAACCTG,5088,...,5179,4438,4677,1495,1257,1433,5486,4717,GTTCAGCGTCGACCAGACACTACCTTATAAGCACTGCAGAACCTGA...,GTTCAGCGTCGACCAGACACTACCTTATAAGCACTGCAGAACCTGA...


In [11]:
filter_df.to_csv('filtered_hek.csv',index=False)

In [12]:
import pandas as pd
import numpy as np

f1 = pd.read_csv("/home/yg2895/media/protein/yg2895/ASO-lib-20250102/DE_analyse/hek-dna-rna.csv")
f2 = pd.read_csv("/home/yg2895/media/scratch/yang/haploinsufficiency/haplo-utr3-oligo-pool_new.csv")

In [13]:
f1.rename(columns={'Unnamed: 0': 'Name'}, inplace=True)

f1.head()

,Name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,AAGAB|ENST00000261880.10|1,2659.245449,0.086427,0.106517,0.811394,4.171394e-01,4.454345e-01
1,AAGAB|ENST00000261880.10|2,2489.885932,0.511739,0.143115,3.575721,3.492642e-04,4.715106e-04
2,AAGAB|ENST00000261880.10|3,6038.426170,0.867749,0.072504,11.968271,5.210315e-33,2.027268e-32
3,AAGAB|ENST00000261880.10|4,2022.130110,0.539307,0.119582,4.509938,6.484647e-06,9.592057e-06
4,AAGAB|ENST00000261880.10|5,3272.032753,0.430862,0.092700,4.647896,3.353381e-06,5.036167e-06


In [14]:
merged_df = pd.merge(f1, f2, on='Name')
merged_df.head()

,Name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,Sequence,sequence
0,AAGAB|ENST00000261880.10|1,2659.245449,0.086427,0.106517,0.811394,4.171394e-01,4.454345e-01,GTTCAGCGTCGACCAATTATTCATACTAGGGTTTGACCAACAAAGA...,GTTCAGCGTCGACCAATTATTCATACTAGGGTTTGACCAACAAAGA...
1,AAGAB|ENST00000261880.10|2,2489.885932,0.511739,0.143115,3.575721,3.492642e-04,4.715106e-04,GTTCAGCGTCGACCATTAGTTTTTAGTAGGAGGTTAAGGAGAAATC...,GTTCAGCGTCGACCATTAGTTTTTAGTAGGAGGTTAAGGAGAAATC...
2,AAGAB|ENST00000261880.10|3,6038.426170,0.867749,0.072504,11.968271,5.210315e-33,2.027268e-32,GTTCAGCGTCGACCATAAATCTCAGAGGTCTGTGTGATTTACTCAA...,GTTCAGCGTCGACCATAAATCTCAGAGGTCTGTGTGATTTACTCAA...
3,AAGAB|ENST00000261880.10|4,2022.130110,0.539307,0.119582,4.509938,6.484647e-06,9.592057e-06,GTTCAGCGTCGACCAATTTGGGTTTTCTCCATTCAAGGAGCTTGTA...,GTTCAGCGTCGACCAATTTGGGTTTTCTCCATTCAAGGAGCTTGTA...
4,AAGAB|ENST00000261880.10|5,3272.032753,0.430862,0.092700,4.647896,3.353381e-06,5.036167e-06,GTTCAGCGTCGACCAGAAAATGCCTCTGGGCATTCTTTTGAAGTAT...,GTTCAGCGTCGACCAGAAAATGCCTCTGGGCATTCTTTTGAAGTAT...


In [15]:
merged_df.drop(columns=["Sequence"], inplace=True)

In [16]:
len(merged_df)

12000

In [17]:
df_filtered = merged_df[~merged_df['sequence'].str.contains('AATAAA|GGGG|ATTAAA')]

# Get the length of the filtered DataFrame
print(len(df_filtered))


3919


In [18]:
df_filtered.to_csv('filtered_data_sk.csv', index=False)